In [41]:
!pip install presto-python-client
import prestodb
import pandas as pd

In [45]:
source_conn=prestodb.dbapi.connect(
    host='presto-service',
    port=8282,
    user='presto',
    catalog='hive',
    schema='source',
)

sample_conn=prestodb.dbapi.connect(
    host='presto-service',
    port=8282,
    user='presto',
    catalog='hive',
    schema='sample',
)



In [43]:
cur = source_conn.cursor()
cur.execute('show tables')
pd.DataFrame.from_records(cur.fetchall(), columns = [i[0] for i in cur.description])

,Table
0,aisles
1,departments
2,order_products
3,order_products_new
4,orders
5,products


In [46]:
cur = sample_conn.cursor()
cur.execute('show tables')
pd.DataFrame.from_records(cur.fetchall(), columns = [i[0] for i in cur.description])

,Table
0,avg_product_count_by_dow
1,order_products
2,product_count
3,stg_order_products
4,stg_orders
5,top_selling_products


In [50]:
cur.execute('select order_id, product_id from stg_order_products where product_id is null limit 10')
pd.DataFrame.from_records(cur.fetchall(), columns = [i[0] for i in cur.description])

,order_id,product_id


In [ ]:
conn=prestodb.dbapi.connect(
    host='presto-service',
    port=8282,
    user='presto',
    catalog='hive',
    schema='sample',
)
cur = conn.cursor()
cur.execute('show tables')
pd.DataFrame.from_records(cur.fetchall(), columns = [i[0] for i in cur.description])

In [53]:
cur.execute('''
with v_orders as (
    select * from  sample.stg_orders
), v_orderprodcts as (
    select  * from sample.stg_order_products
), v_products as (
    select * from  source.products
), v_aisles as (
    select * from  source.aisles
), v_departments as (
    select * from  source.departments
)  select 
    t1.order_id, t1.user_id, t1.order_number, t1.order_dow, t1.order_hour_of_day, t1.days_since_prior_order, t1.days_since_prior_order_cum, t1.order_date
    , t2.product_id, t2.add_to_cart_order, t2.reordered
    
    from v_orders t1
    join v_orderprodcts t2 on t1.order_id = t2.order_id
    
''')
pd.DataFrame.from_records(cur.fetchall(), columns = [i[0] for i in cur.description])

,order_id,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,days_since_prior_order_cum,order_date,product_id,add_to_cart_order,reordered


In [ ]:
cur.execute('''select  * from sample.stg_orders where order_id = 1569900''')
pd.DataFrame.from_records(cur.fetchall(), columns = [i[0] for i in cur.description])

In [ ]:
cur.execute('''select  * from sample.stg_order_products where order_id = 1569900''')
pd.DataFrame.from_records(cur.fetchall(), columns = [i[0] for i in cur.description])